In [0]:
import numpy as np

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive


To make sure that every single line will be  printed, even if they're in the same cell, we can use thf ollowing config:

In [0]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

Build a directory and name it *ML_Python_PMCRT* in your colab directory in google drive. Then upload the two files "CCLE_ExpMat_Top500Genes.csv" and "CCLE_ExpMat_Pheno.csv" in this directory.

In [0]:
import pandas as pd

CCLE_exp = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ML_Python_PMCRT/CCLE_ExpMat_Top500Genes.csv', index_col=0)
CCLE_pheno = pd.read_csv('/content/gdrive/My Drive/Colab Notebooks/ML_Python_PMCRT/CCLE_ExpMat_Pheno.csv', index_col=0)
####
print("shape of the expression dataframe:", CCLE_exp.shape)
print("shape of the expression dataframe:", CCLE_pheno.shape)
####
CCLE_exp = CCLE_exp.transpose()
n_samples, n_features = CCLE_exp.shape


CCLE_exp.shape

shape of the expression dataframe: (500, 550)
shape of the expression dataframe: (550, 17)


(550, 500)

In [0]:
import collections
collections.Counter(CCLE_pheno.loc[:,"tissueid"])

Counter({'breast': 54,
         'central_nervous_system': 49,
         'haematopoietic_and_lymphoid_tissue': 170,
         'large_intestine': 57,
         'lung': 172,
         'skin': 48})

In [0]:
type(np.where(CCLE_pheno.loc[:,"tissueid"] == 'breast'))
type(np.where(CCLE_pheno.loc[:,"tissueid"] == 'breast')[0])

tuple

numpy.ndarray

In [0]:
collections.Counter(np.where(CCLE_pheno.loc[:,"tissueid"] == 'breast')[0])

We need to convert the labels, tissue names, to numbers.

In [0]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

le.fit(CCLE_pheno.loc[:,"tissueid"])
y=le.transform(CCLE_pheno.loc[:,"tissueid"])

collections.Counter(y)

LabelEncoder()

Counter({0: 54, 1: 49, 2: 170, 3: 57, 4: 172, 5: 48})

In [0]:
CCLE_pheno['Encoded_tissue'] = y
CCLE_pheno.columns.values

array(['cellid', 'tissueid', 'CCLE.cellid', 'link', 'CCLE.name',
       'Cell.line.primary.name', 'Cell.line.aliases', 'Gender',
       'Site.Primary', 'Histology', 'Hist.Subtype1', 'Notes', 'Source',
       'Expression.arrays', 'SNP.arrays', 'Oncomap',
       'Hybrid.Capture.Sequencing', 'Encoded_tissue'], dtype=object)

# Types of observations

There are three types of data measurements or variables we deal with in building machine learning models.

* **Nominal**:
*Nominal* (categorical or qualitative) variables or observations are labels without any order or quantitative difference between the label classes. Examples of nominal data are tissue type, healthy versus malignant tissue, male versus female, etc.
* **Ordinal**:
For *ordinal* variables, order of the values assigned (or tested) for each sample is important.  Examples of ordinal data are tumor stage, level of satisfaction about a product, etc.
* **Interval**:
For *Interval* variables, both order and exact difference between the data points matter. Examples of ordinal data are tumor size and age.

# Building classification models

Among the available classification methods in Python, we focus on the following five to build classification models of tissue type of the cancer cell lines in our dataset:

* Logistic regression
* K- nearest neighbour
* Naive Bayes
* Random forest
* Support vector machine

## Logistic regression
If we have set of features X1 to Xn, y can be obtained as:
\begin{equation*} y=b0+b1X1+b2X2+...+bnXn\end{equation*}

where y is the predicted value obtained by weighted sum of the feature values.

Then probability of each class (for example tissue class BREAST) can be obtained using the logistic function 

\begin{equation*} p(class=BREAST)=\frac{1}{(1+exp(-y))} \end{equation*}

Based on the given class labels and the features given in the trainign data, coefficients b0 to bn can be ontained during the optimization process.

b0 to bn are fixed for all samples while X1 to Xn are feature values specific to each sample. Hence, the logistic function will give us probability of each class assigned to each sample. Finally, the model will choose the class with the highest probability for each sample.


**Note.** The logistic regression model is parametric and the parameters are the regression coefficiets b0 to bn.


In [0]:
from sklearn.linear_model import LogisticRegression as LR

# Initialize our classifier
#logreg = LogisticRegression()
logreg = LR()

# Fitting the model with the data
logreg.fit(CCLE_exp, y)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

In [0]:
y_pred = logreg.predict(CCLE_exp)
print(y_pred)

Checking the accuracy of the model

In [0]:
from sklearn import metrics

print("accuracy of the predictions:", metrics.accuracy_score(y,y_pred))
print("precision of the predictions:", metrics.precision_score(y,y_pred, average="macro"))
print("blanced accuracy of the predictions:", metrics.balanced_accuracy_score(y, y_pred))
print("MCC of the predictions:", metrics.matthews_corrcoef(y, y_pred))
print("Confusion matrix of the predictions:", metrics.confusion_matrix(y, y_pred))

accuracy of the predictions: 1.0
precision of the predictions: 1.0
blanced accuracy of the predictions: 1.0
MCC of the predictions: 1.0
Confusion matrix of the predictions: [[ 54   0   0   0   0   0]
 [  0  49   0   0   0   0]
 [  0   0 170   0   0   0]
 [  0   0   0  57   0   0]
 [  0   0   0   0 172   0]
 [  0   0   0   0   0  48]]


## Splitting data to training and testing sets

To investigate performance of our model, we need to split the data to training and testing sets. This will help us to check potential overfitting in our model training.

In [0]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(CCLE_exp, y, test_size=0.4, random_state=5)

Training a logistic regression model on the training set:

In [0]:
logreg = LogisticRegression()

# Fitting the model with the data
logreg.fit(X_train, y_train)

/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/linear_model/logistic.py:460: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='warn',
          n_jobs=None, penalty='l2', random_state=None, solver='warn',
          tol=0.0001, verbose=0, warm_start=False)

Testing the model on the testing set:

In [0]:
y_pred_test = logreg.predict(X_test)
print(y_pred_test)

# Performance measures

To assess performance of the machine learning model, we can use one or few of the following measures of the performance of the model:

* **Accuracy**: This measure gives you a sense of performance for all the classes together as follows:

\begin{equation*} Accuracy=\frac{Number\:of\:correct\:predictions}{(Total\:number\:of\:data\:points (samples))} \end{equation*}

In case of binary classification, considering two groups of positive and negative samples, accuracy can be re-written as follows:

\begin{equation*} Accuracy=\frac{TP+TN}{(TP+TN+FP+FN)} \end{equation*}

Note. In case of imbalance in your classes, even a terrible classifier may result in high accuracy. As an example, assume you have 100 samples, 80 positive and 20 negative. Accuracy of a model that predict every sample is positive could be 0.8:

\begin{equation*} Accuracy=\frac{80+0}{(80+0+20+0)} = 0.8 \end{equation*}

which can be a model with zero weight for all the features and a positive intercept (constant in the model).

* **Precision**: Precision cares about true classification of a target class and is defined as follows:

\begin{equation*} Precision=\frac{TP}{(TP+FP)} \end{equation*}

As you may have noticed, the same issue holds for precision in case of imbalanced data. The precision of the same example model will be 0.8.

* **Recall (or sensitivity)**: Recall tells you the probability of retreival of target population using the develped model and is defined as follows: 

\begin{equation*} Recall=\frac{TP}{(TP+FN)} \end{equation*}

Considering the same example, recall of the model with only an intercept term could be equal to 1:

\begin{equation*} Recall=\frac{80}{(80+0)} \end{equation*}

* **Specificity (or selectivity)**:  Specificity is define

\begin{equation*} Specificity=\frac{TN}{(TN+FP)} \end{equation*}

Specificity of the example model will be equal to 0 which could help us to figure out that the given model was not a good one.

* **Balanced accuracy**: Balanced accuracy is difned as the average of the recall and specificity:


\begin{equation*} Balanced\:accuracy=\frac{Recall+Specificity}{2}\end{equation*}

Hence, the balanced accuracy of the example model will be 0.5.

* **Confusion matrix (or error matrix)**: True and false classification of the samples in all the classes can be shown in a matrix which is called confusion (or error) matrix. The columsn are usually considered as the actual classes and rows as predicted classes. Hence, the diagonal elements of the matrix will be the total number of true classifcation in each class. 

* **Matthews correlation coefficient (MCC)**: MCC is one of the best meaures to summarize the confusion matrix with a little bit more complex definitions as follows: 

\begin{equation*} MCC=\frac{TP*TN-FP*FN}{\sqrt{(TP+FP)(TP+FN)(TN+FP)(TN+FN)}}\end{equation*}


In [0]:
print("accuracy of the predictions:", metrics.accuracy_score(y_test, y_pred_test))
print("blanced accuracy of the predictions:", metrics.balanced_accuracy_score(y_test, y_pred_test))
print("MCC of the predictions:", metrics.matthews_corrcoef(y_test, y_pred_test))
print("Confusion matrix of the predictions:", metrics.confusion_matrix(y_test, y_pred_test))

accuracy of the predictions: 0.9409090909090909
blanced accuracy of the predictions: 0.9147621019488467
MCC of the predictions: 0.9226665524020982
Confusion matrix of the predictions: [[15  0  0  0  1  0]
 [ 1 16  0  0  2  1]
 [ 0  0 65  0  1  2]
 [ 0  0  0 23  2  0]
 [ 0  0  0  1 72  0]
 [ 0  0  1  0  1 16]]


# K nearest neighbour(k-NN)

K nearest neighbour uses a distance metric like Euclidean distance to identity similarity of target data point (sample) in test or validation set to the data points (samples) in the trainign set. Then based on the user specified k, it finds the k closest points (samples) to the target data point. Afterward, it chooses the most frequent label among the k closes points (majority voting) as the class label of the target sample. The class labels can be also assigned based on weighted voting of the k closest data points to the data point.

**Note.** The k nearest neighbour is non-parametric.


In [0]:
from sklearn.neighbors import KNeighborsClassifier

# Initialize our classifier
knn = KNeighborsClassifier(n_neighbors=10)

# Fitting the model with the data
knn.fit(CCLE_exp, y)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=10, p=2,
           weights='uniform')

In [0]:
y_pred = knn.predict(CCLE_exp)
print(y_pred)

[2 1 5 1 5 4 5 2 2 4 4 1 2 0 4 2 2 0 0 1 2 3 5 2 4 4 0 4 4 4 4 4 0 1 1 4 2
 2 4 4 3 5 5 5 5 5 4 4 4 4 3 4 1 2 2 1 2 1 4 4 4 4 4 2 4 3 0 2 4 2 2 4 5 1
 1 2 1 1 3 1 4 0 1 0 0 0 4 4 0 0 4 0 3 3 1 2 2 2 4 4 0 2 5 3 3 2 4 5 5 2 2
 2 1 2 2 2 2 2 3 2 2 1 4 1 1 2 2 2 2 2 4 4 4 2 3 4 2 3 3 3 3 4 4 4 5 2 0 0
 1 0 4 0 0 5 0 2 5 5 5 2 2 2 2 2 2 2 2 2 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 3 4 4 4 4 4 4 4 4 4 4 4 3 4 4 4 4 1 2 2 1 2 2 2 4 2 2 3 2 4 3 2 1 2 2 2 4
 4 1 1 1 1 5 4 2 3 4 2 5 5 5 5 5 5 4 2 1 3 2 2 2 2 1 3 3 3 4 1 3 3 3 4 3 0
 3 1 2 2 1 1 2 1 5 5 0 0 5 1 1 0 1 2 2 2 2 0 3 2 4 3 3 3 3 3 5 4 4 4 4 4 4
 2 2 4 0 2 2 2 1 2 3 4 4 4 0 4 4 0 3 4 0 4 4 4 4 4 4 4 3 4 4 4 4 2 4 4 5 2
 0 5 0 5 0 5 0 2 5 0 4 1 0 5 5 0 0 5 0 5 0 5 5 5 5 5 3 2 2 5 2 4 2 2 2 2 2
 5 2 2 2 2 2 1 2 2 2 2 2 2 2 2 2 2 1 2 0 2 2 1 5 4 3 4 1 2 4 1 2 2 2 2 4 2
 2 2 2 2 2 2 4 2 2 2 2 4 4 4 4 3 4 4 4 4 4 4 1 4 4 4 4 4 4 4 4 4 4 4 4 4 4
 3 2 2 2 2 2 2 2 2 2 3 2 

In [0]:
print("accuracy of the predictions:", metrics.accuracy_score(y, y_pred))
print("blanced accuracy of the predictions:", metrics.balanced_accuracy_score(y, y_pred))
print("MCC of the predictions:", metrics.matthews_corrcoef(y, y_pred))
print("Confusion matrix of the predictions:", metrics.confusion_matrix(y, y_pred))

accuracy of the predictions: 0.9018181818181819
blanced accuracy of the predictions: 0.8687430557244885
MCC of the predictions: 0.8732937382402157
Confusion matrix of the predictions: [[ 37   3   0   1  13   0]
 [  0  44   0   0   3   2]
 [  0   1 160   0   2   7]
 [  0   1   0  49   4   3]
 [  2   2   0   4 164   0]
 [  3   1   0   0   2  42]]


In [0]:
knn = KNeighborsClassifier()

# Fitting the model with the data
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=None, n_neighbors=5, p=2,
           weights='uniform')

Testing the model on the testing set:

In [0]:
y_pred_test = knn.predict(X_test)
print(y_pred_test)

[4 0 2 3 4 0 3 2 2 2 2 0 3 4 4 4 2 2 4 4 4 2 1 4 1 0 2 5 4 0 4 2 5 1 2 4 0
 4 2 2 1 3 4 4 2 2 2 0 4 2 2 4 2 2 4 4 1 4 2 2 4 4 4 4 5 3 0 0 0 3 5 2 3 3
 5 2 4 4 4 2 4 4 3 0 3 4 1 4 4 4 2 4 4 4 5 2 5 2 1 4 4 4 4 2 2 3 2 4 0 2 4
 1 2 5 2 2 2 4 4 2 1 4 3 4 5 2 5 1 4 1 0 2 2 4 0 2 3 2 1 4 2 4 2 4 2 4 2 5
 3 3 3 5 2 2 2 2 4 4 2 4 3 4 3 5 4 2 4 4 4 3 0 0 4 2 2 2 2 4 2 2 4 2 4 2 2
 4 3 2 2 4 4 4 1 3 1 5 4 0 4 5 4 4 0 4 2 4 2 5 3 4 5 3 1 4 5 2 2 2 3 1]


In [0]:
print("accuracy of the predictions:", metrics.accuracy_score(y_test, y_pred_test))
print("blanced accuracy of the predictions:", metrics.balanced_accuracy_score(y_test, y_pred_test))
print("MCC of the predictions:", metrics.matthews_corrcoef(y_test, y_pred_test))
print("Confusion matrix of the predictions:", metrics.confusion_matrix(y_test, y_pred_test))

accuracy of the predictions: 0.9090909090909091
blanced accuracy of the predictions: 0.8755370892649297
MCC of the predictions: 0.8804643029362605
Confusion matrix of the predictions: [[13  0  0  0  3  0]
 [ 0 16  0  0  3  1]
 [ 0  0 65  0  1  2]
 [ 1  0  0 23  1  0]
 [ 3  0  1  1 68  0]
 [ 1  0  2  0  0 15]]


# Naive Bayes
To understand Naive Bayes algotirhm, we need to know what Bayes theorem. Bayes theorem related conditional rpobabilities as follows:

\begin{equation*} p(A|B)p(B)=p(B|A)p(A) \end{equation*}
that can be rewritten as

\begin{equation*} p(A|B)=\frac{p(B|A)p(A)}{p(B)} \end{equation*}

where p(A) and p(B) are probabilities of events A and B, respectively. p(A|B) and p(B|A) are also conditional probabilities of A given B and B given A, respectively.
**Example without numbers**

Now let's assume we have 3 features X1, X2 and X3 and we want to identify the probability of class C for sample A with feature values *x1*, *x2* and *x3*:

\begin{equation*} p(class=C|X1=x1, X2=x2 , X3=x3)=\frac{p(X1=x1|class=C)p(X2=x2|lass=C)p(X3=x3|class=C)p(class=C)}{p(X1=x1)p(X2=x2)p(X3=x3)} \end{equation*}

where 
\begin{equation*} p(X1=x1, X2=x2 , X3=x3)=p(X1=x1)p(X2=x2)p(X3=x3) \end{equation*}
and
\begin{equation*} p(X1=x1, X2=x2 , X3=x3|class=C)=p(X1=x1|class=C)p(X2=x2|class=C)p(X3=x3|lass=C)p(class=C) \end{equation*}

as the features are independent variables. 

**Real life example with numbers**
We want to know the chance of having breast cancer if the diagnosis test is positive for a woman with the age between 40 and 60. This example is mainly for understanding Bayes theorem not Naive Bayes classifier. In case of Naive Bayes algorithm, this process can be easily extended to multiple features as described in the above example.

***Assumptions (not necessarily correct)***
* 2% of women between 40 and 60 have breast cancer
* True positive rate is 95% (if a woman has breast cancer, it will be diagnosed with 95% probability). Therefore, 5% of the time the women without breast cancer will be diagnosed positively by the test.

Now the question is *What is the chance of havign breast cancer if a woman has positive result from a diagnosis test?*

\begin{equation*} p(having \quad breast \quad cancer|positive)=\frac{p(positive|breast \quad cancer)p(breast cancer)}{p(positive)} \end{equation*}

where 


\begin{equation*} p(positive) = p(positive|having \quad breast \quad cancer)p(having \quad breast \quad cancer) \\+ p(positive|not \quad having \quad breast \quad cancer)p(not \quad having \quad breast \quad cancer)\\=
0.95*0.02+0.05*0.98\\=0.068\end{equation*}

Therefore,

\begin{equation*} p(having \quad breast  \quad cancer|positive)=\frac{p(positive|breast \quad cancer)p(having \quad breast \quad cancer)}{p(positive)}\\= \frac{0.95*0.02}{0.068}\\=0.28\end{equation*}


As we can see, there is only 28% chance of having cancer upon positive test result. Although the numbers were not clinically valid numbers, we deal with similar results in disease diagnosis. This is one of the reasons that further checkups by phycisions are mandatory upon positive results. Do not panic when you have a positive result but follow up with your doctor immediately.

**Note.** Naive Bayes classifier is called ***Naive*** as it assumes each feature will independently contribute in prediction of a class for each data point (sample).

In [0]:
from sklearn.naive_bayes import GaussianNB

# Initialize our classifier
gnb = GaussianNB()

# Train our classifier
model = gnb.fit(CCLE_exp, y)

In [0]:
y_pred = knn.predict(CCLE_exp)
print(y_pred)

In [0]:
print("accuracy of the predictions:", metrics.accuracy_score(y, y_pred))
print("blanced accuracy of the predictions:", metrics.balanced_accuracy_score(y, y_pred))
print("MCC of the predictions:", metrics.matthews_corrcoef(y, y_pred))
print("Confusion matrix of the predictions:", metrics.confusion_matrix(y, y_pred))

In [0]:
gnb = GaussianNB()

# Fitting the model with the data
gnb.fit(X_train, y_train)

Testing the model on the testing set:

In [0]:
y_pred_test = knn.predict(X_test)
print(y_pred_test)

In [0]:
print("accuracy of the predictions:", metrics.accuracy_score(y_test, y_pred_test))
print("blanced accuracy of the predictions:", metrics.balanced_accuracy_score(y_test, y_pred_test))
print("MCC of the predictions:", metrics.matthews_corrcoef(y_test, y_pred_test))
print("Confusion matrix of the predictions:", metrics.confusion_matrix(y_test, y_pred_test))